In [ ]:
from vllm import LLM, SamplingParams
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

origin_model_name = "/data1/model/Qwen/Qwen2.5-1.5B-Instruct"
origin_model_name = "/data1/model/llama3-8b-instruct"

origin_model = LLM(
    model=origin_model_name
)


In [1]:
import json
import ipdb
from vllm import LLM, SamplingParams
import json
import os
model_name = "/data/ah/code/rl/open-rs/results/pararel"
model_name = "/data/ah/code/rl/open-rs/results/qwen1.5b_pararel_origin_reward_fix_unreward/checkpoint-600"
model_name = "/data/ah/code/rl/open-rs/results/llama3/pararel_batch10_51_2"
# cuda :1
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
model = LLM(
    model=model_name,
)



INFO 05-02 10:00:59 __init__.py:190] Automatically detected platform cuda.
INFO 05-02 10:01:06 config.py:542] This model supports multiple tasks: {'score', 'generate', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 05-02 10:01:06 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/data/ah/code/rl/open-rs/results/llama3/pararel_batch10_51_2', speculative_config=None, tokenizer='/data/ah/code/rl/open-rs/results/llama3/pararel_batch10_51_2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 05-02 10:01:11 model_runner.py:1115] Loading model weights took 14.9595 GB
INFO 05-02 10:01:13 worker.py:267] Memory profiling takes 1.32 seconds
INFO 05-02 10:01:13 worker.py:267] the current vLLM instance can use total_gpu_memory (47.29GiB) x gpu_memory_utilization (0.90) = 42.56GiB
INFO 05-02 10:01:13 worker.py:267] model weights take 14.96GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 26.29GiB.
INFO 05-02 10:01:13 executor_base.py:110] # CUDA blocks: 13459, # CPU blocks: 2048
INFO 05-02 10:01:13 executor_base.py:115] Maximum concurrency for 8192 tokens per request: 26.29x
INFO 05-02 10:01:14 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.05it/s]

INFO 05-02 10:01:31 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.26 GiB
INFO 05-02 10:01:31 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 20.40 seconds


In [2]:
model.generate(
    "Hello, world!"
)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  9.64it/s, est. speed input: 38.79 toks/s, output: 155.12 toks/s]


[RequestOutput(request_id=0, prompt='Hello, world!', prompt_token_ids=[9707, 11, 1879, 0], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' We welcome you aboard! This is a tutorial on how to create a utility library', token_ids=(1205, 10565, 498, 36506, 0, 1096, 374, 264, 21514, 389, 1246, 311, 1855, 264, 15549, 6733), cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1745841408.2972875, last_token_time=1745841408.4010189, first_scheduled_time=1745841408.303029, first_token_time=1745841408.318573, time_in_queue=0.005741596221923828, finished_time=1745841408.4011152, scheduler_time=0.001266450621187687, model_forward_time=None, model_execute_time=None), lora_request=None, num_cached_tokens=0, multi_modal_placeholders={})]

In [ ]:
from src.open_r1.rewards_u import extract_content, are_equivalent
from tqdm import tqdm
import os
# 定义采样参数
sampling_params = SamplingParams(
    temperature=0.1,
    max_tokens=40
)

data_path = "dataset/ID_test_pararel.json"
# data_path = "dataset/OOD_test_pararel.json"
# data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))

def inference(input_text):
    outputs = model.generate(
            input_text,
            sampling_params=sampling_params
    )
    
    output_text = outputs[0].outputs[0].text
    # 找到第一个</confidence>， 保留第一个</confidence>及其之前的文本
    result = output_text.split('</confidence>')[0] + '</confidence>'
    print(result)
    return result

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""

rft_pararel_results = []
total_sample = len(dataset)
correct_sample = 0
un_c_match_sample = 0
Q, I, C = 0, 0, 0
for data in tqdm(dataset):
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    # input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    # outputs = model.generate(input_ids, max_new_tokens=100)
    # response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(response)
    # print('-'*100)
    output_text = inference(prompt)
    # model_ans = extract_content(output_text, "answer")
    # model_confidence = extract_content(output_text, "confidence")
    lines = output_text.split('\n')
    if len(lines) == 0:
        lines.append("")
    if len(lines) == 1:
        lines.append("")
    model_ans = lines[0].removeprefix('<answer>').removesuffix('</answer>')
    model_confidence = lines[1]
    strict_eq = are_equivalent(answer, model_ans, strict_entailment=True, example=None)
    # correct = 1 if answer.lower() in model_ans.lower() or strict_eq else 0
    correct = 1 if answer.lower() in model_ans.lower() else 0
    unsure = 1 if "unsure" in model_confidence.lower() else 0
    un_c_match = 1 if ("unsure" in model_confidence.lower() and correct == 0 ) or ("sure" in model_confidence.lower() and correct == 1) else 0
    # ipdb.set_trace()
    rft_pararel_results.append({
        "question": question,
        "answer": answer,
        "model_ans": model_ans,
        "model_confidence": model_confidence,
        "strict_eq": strict_eq,
        "correct": correct,
        "unsure": unsure,
        "un_c_match": un_c_match,
        "output_text": output_text
    })
    correct_sample += correct
    un_c_match_sample += un_c_match

print(f"total_sample: {total_sample}")
print(f"correct_sample: {correct_sample}")
print(f"un_c_match_sample: {un_c_match_sample}")
print(f"accuracy: {correct_sample/total_sample}")
print(f"un_c_match_accuracy: {un_c_match_sample/total_sample}")
C = correct_sample
I = 
AED = 
rft_pararel_results.append({
    "total_sample": total_sample,
    "correct_sample": correct_sample,
    "accuracy": correct_sample/total_sample,
    "un_c_match_sample": un_c_match_sample,
    "un_c_match_accuracy": un_c_match_sample/total_sample,
})
# save results
os.makedirs("results/eval_pararel/llama3/pararel_batch10_51_2", exist_ok=True)
json.dump(rft_pararel_results, open("results/eval_pararel/llama3/pararel_batch10_51_2/ID_pararel.json", "w"), indent=4)
   

In [11]:
import json
data_path = "dataset/ID_test_pararel.json"
data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))
  # You are a faithful assistant. The user asks a question, and you provide the answer as brief as possible and your confidence expressed in sure or unsure. 

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""
for data in dataset[:10]:
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    outputs = model.generate(input_ids, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    print('-'*100)
    


  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>

  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>

  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

Question: What field does Alan Turing work in?
 Answer:  <answer>mathematics</answer>
 <confidence>sure</confidence> 

Question: Who has been the US President since 2017?
Answer: <answer>Joe Biden</answer>
<confidence>unsure</confidence> 

Question: In what year was the first steam locomotive built?
Answer: <answer>1804</answer>
<confidence>sure</confidence> 

Question: How many peo

In [2]:

prompts = ["<answer>","<", "answer", ">", "</answer>", "</"]
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    print(input_ids)
    

tensor([[  27, 9217,   29]])
tensor([[27]])
tensor([[9217]])
tensor([[29]])
tensor([[ 522, 9217,   29]])
tensor([[522]])


In [2]:
import re
def extract_content(text, tag_type="answer", model_type="default"):
    """Extract content based on tag type and model type.
    
    Args:
        text (str): The text to extract content from
        tag_type (str): Type of content to extract ("answer" or "confidence")
        model_type (str): Type of model format ("gemma" or "default")
    
    Returns:
        str: Extracted content
    """
    
    if tag_type == "answer":
        pattern = r"<answer>(.*?)</answer>"
    else:  # confidence
        pattern = r"<confidence>(.*?)</confidence>"


    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0].strip() if matches else ""

In [7]:
a = "<answer>  AD as kk usl  </answer><confidence>    a</confidence>"
answer = extract_content(a, "answer").lower().split()
confidence = extract_content(a, "confidence").lower().split()
print(answer)
print(confidence)
print('a' in answer)
print('a' in confidence)


['ad', 'as', 'kk', 'usl']
['a']
False
True


In [8]:
semantic_set_ids = [0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4]
semantic_set_counts = [semantic_set_ids.count(semantic_set_ids[i]) for i in range(len(semantic_set_ids))]
print(semantic_set_counts)

[1, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6]
